In [1]:
import os
import sys
sys.path.append('/scratch/programming/preprocessing-pipeline/src')
import numpy as np
import shutil
from skimage import io
from tqdm import tqdm
from atlas.NgSegmentMaker import NgConverter
from lib.sqlcontroller import SqlController
from lib.file_location import FileLocationManager
from lib.utilities_process import test_dir, SCALING_FACTOR
from lib.utilities_mask import place_image
import scipy

In [11]:
animal = 'DK55'
sqlController = SqlController(animal)
fileLocationManager = FileLocationManager(animal)
INPUT = fileLocationManager.aligned_rotated_and_padded_thumbnail_mask
error = test_dir(animal, INPUT, downsample=True, same_size=True)
if len(error) > 0:
    print(error)
    sys.exit()

OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'shell')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(os.listdir(INPUT))
len_files = len(files)
midpoint = len_files // 2
resolution = sqlController.scan_run.resolution
resolution = int(resolution * 1000 / SCALING_FACTOR)


In [18]:
volume = []
for file in tqdm(files):
    tif = io.imread(os.path.join(INPUT, file))
    volume.append(tif)
ids = np.unique(volume)
ids = [(i,i) for i in ids]
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)


100%|██████████| 485/485 [00:04<00:00, 117.16it/s]


In [19]:
fvolume = scipy.ndimage.gaussian_filter(volume,sigma=1.5)
fvolume = (fvolume!=0).astype('uint8')
fvolume = np.pad(fvolume,[10,10,10])
ng = NgConverter(fvolume, [resolution, resolution, 20000], offset=[0,0,0])
ng.create_neuroglancer_files(OUTPUT_DIR,ids)

Uploading:  51%|█████▏    | 3793/7400 [00:49<12:11,  4.93it/s] 

In [8]:
volume.shape

(2359, 1546, 485)